In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.metrics import accuracy_score

# Assuming you have the dataset on your Google Drive and you have mounted it
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/sign-language'

Mounted at /content/drive


In [ ]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Load the dataset
dataset = torchvision.datasets.ImageFolder(root=data_path, transform=transform)

In [ ]:
# Split the data into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)
num_classes = len(dataset.classes)
vgg16.classifier[6] = nn.Linear(4096, num_classes)  # Modify the classifier for your number of classes

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 240MB/s]


In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss / len(train_loader)}")

Epoch [1/10] - Loss: 0.06995313724627542
Epoch [2/10] - Loss: 0.027467800688970347
Epoch [3/10] - Loss: 0.020163138133838125
Epoch [4/10] - Loss: 0.01670549755900815
Epoch [5/10] - Loss: 0.010231852091079496
Epoch [6/10] - Loss: 0.007694953827645488
Epoch [7/10] - Loss: 0.004842451944568893
Epoch [8/10] - Loss: 0.004107699921049694
Epoch [9/10] - Loss: 0.0031243393076811512
Epoch [10/10] - Loss: 0.0017411516977438662


In [ ]:

# Calculate accuracy on test set
vgg16.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = vgg16(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 94.84%
